In [1]:
! pip install langchain langchain-openai langchain-milvus pymilvus


  Using cached langgraph_sdk-0.2.9-py3-none-any.whl.metadata (1.5 kB)
  Using cached xxhash-3.6.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (13 kB)
Using cached langgraph_sdk-0.2.9-py3-none-any.whl (56 kB)
Using cached xxhash-3.6.0-cp311-cp311-macosx_11_0_arm64.whl (30 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [langchain]


In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_chunk_pdf(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    print("Preview of first page:\n", documents[0].page_content[:500])

    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=1000,
        chunk_overlap=50
    )
    chunks = splitter.split_documents(documents)
    print(f"Total chunks created: {len(chunks)}")
    return chunks

In [ ]:
chunks = load_and_chunk_pdf(r"your_file_path/sample.pdf")
print(chunks)

Preview of first page:
 Domestic Travel Policy 
 
 
 
 
www.yash.com 
 
 
 
 
 
 
 

 This document is exclusive property of YASH Technologies Pvt. Ltd. 
No part or whole of this document may be reproduced in any form without written permission.
Total chunks created: 15
[Document(metadata={'producer': 'www.ilovepdf.com', 'creator': 'Microsoft® Word 2016', 'creationdate': '2023-09-13T07:16:42+00:00', 'title': 'YASH-HRO-P002', 'author': 'Human Resource', 'subject': 'Pol Domestic Travel v2.5', 'keywords': 'ver_3.0', 'moddate': '2023-09-13T07:16:44+00:00', 'source': 'C:/Users/rimzim.malviya/Documents/Gen AI Practices/Handson-practice/Gen-AI-Capstone-Projects/notebooks/Domestic Travel Policy.pdf', 'total_pages': 15, 'page': 0, 'page_label': '1'}, page_content='Domestic Travel Policy \n \n \n \n \nwww.yash.com \n \n \n \n \n \n \n \n\uf020\n\uf0e3 This document is exclusive property of YASH Technologies Pvt. Ltd. \nNo part or whole of this document may be reproduced in any form without wri

In [5]:
! pip install python-dotenv


In [8]:
import os
from dotenv import load_dotenv
load_dotenv()

AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_VERSION = os.getenv("AZURE_OPENAI_VERSION")

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings

def initialize_embedding_model():
    embeddings = AzureOpenAIEmbeddings(
        deployment="text-embedding-3-small",
        model="text-embedding-3-small",
        openai_api_type="azure",
        openai_api_key="your_api_key",
        azure_endpoint="your_endpoint",
        openai_api_version="2024-02-15-preview",
        chunk_size=2048
    )
    print("Embeddings model initialized.")
    return embeddings

embeddings = initialize_embedding_model()
print(embeddings)


Embeddings model initialized.
client=<openai.resources.embeddings.Embeddings object at 0x000002449D4D8ED0> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000002449D949750> model='text-embedding-3-small' dimensions=None deployment='text-embedding-3-small' openai_api_version='2024-02-15-preview' openai_api_base=None openai_api_type='azure' openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=2048 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True azure_endpoint='https://sanje-mhx7nb8u-northcentralus.cognitiveservices.azure.com/' azure_ad_token=None azure_ad_token_provider=None azure_ad_async_token_provider=

In [14]:
from pymilvus import connections
from pymilvus import connections, utility, CollectionSchema, FieldSchema, DataType, Collection
collection_name = "domestic_travel_policy_embeddings"
dim = 1536  # Dimension of the embedding vectors
MILVUS_URI = os.getenv("MILVUS_URI")
MILVUS_TOKEN = os.getenv("MILVUS_TOKEN")


In [ ]:
def ensure_milvus_collection(collection_name, dim, uri, token):
    connections.connect(uri=uri, token=token)
    if utility.has_collection(collection_name):
        print(f"Collection '{collection_name}' already exists. Reusing it.")
        return  # Skip creation if it exists
    print(f"Creating new collection '{collection_name}'...")
    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=dim)
    ]
    schema = CollectionSchema(fields, description="PDF vector collection")
    Collection(name=collection_name, schema=schema)
    print("Collection created.")

ensure_milvus_collection(collection_name, dim=1536, uri=MILVUS_URI, token=MILVUS_TOKEN)

Collection 'domestic_travel_policy_embeddings' already exists. Reusing it.


In [ ]:
from langchain_milvus import Milvus
def store_in_milvus(chunks, embeddings, collection_name="domestic_travel_policy_embeddings"):

    vector_db = Milvus.from_documents(
        chunks,
        embeddings,
        collection_name="travel_policy_embeddings",
        connection_args={
            "uri": MILVUS_URI,
            "token": MILVUS_TOKEN",
            "secure": True
        },
        drop_old=True
    )
    return vector_db

vector_db = store_in_milvus(chunks, embeddings, collection_name=collection_name)

In [ ]:
from langchain_milvus import Milvus
vector_db = Milvus(
    embedding_function=embeddings,
    collection_name="travel_policy_embeddings",   # must match the collection you created
    connection_args={
        "uri": MILVUS_URI,
        "token": MILVUS_TOKEN,
        "secure": True
    },
    text_field="text"
)

query = "What are Grade or Employee Type (Per day) limits for travel expenses?"
results = vector_db.similarity_search(query, k=3)  # top 3 results

for i, doc in enumerate(results, 1):
    print(f"Result {i}:\n{doc.page_content[:300]}\n")

Result 1:
Domestic Travel Policy                                      
 
  
 
 
 YASH Confidential Page 8 of 15 
3rd A.C Lower of actual or ₹ 200/- 
2nd A.C Lower of actual or ₹ 300/- 
1st A.C Not Applicable 
7.1.4 GROUP TRAVEL:  
 
In a scenario when a team composed of employees from different grades travels

Result 2:
Domestic Travel Policy                                      
 
  
 
 
 YASH Confidential Page 12 of 15 
 Up-gradation & expensive travel ticket.  
 Cost related to travel of extra member other than employee himself/herself.  
 Extra baggage cost other than standard airlines policy. 
 Travel Insu

Result 3:
Domestic Travel Policy                                      
 
  
 
 
 YASH Confidential Page 5 of 15 
 
5.0 OVERVIEW 
 
5.1 For business reasons, employees may be required to travel within India.  
 
 For employees who travel outside their base location on business purpose; Company provides them t



In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain.chains import RetrievalQA
llm = AzureChatOpenAI(
    azure_endpoint="your_endpoint",
    openai_api_key="your_api_key",
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-4o",
    temperature=0.7
)
llm.invoke("Hello, world!")


AIMessage(content='Hello, world! 😊 How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 11, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'id': 'chatcmpl-CdGp1Lyzr6jKXTLyVjBmJGJGM6XKY', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 

In [22]:
from langchain.prompts import PromptTemplate

prompt_template = """You are a helpful assistant that provides accurate information based on the provided document excerpts.
When answering questions, refer only to the information contained in the excerpts below.
If the information is not present in the excerpts, respond with 'I don't know'.

Context:
{context}

Question:
{question}

Answer:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)


In [23]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_db.as_retriever(),
    chain_type="stuff",   # or "map_reduce"
    chain_type_kwargs={"prompt": prompt}
)


In [24]:
response = qa_chain.invoke({"query": "What are travel allowance for E3 grade employees?"})
print(response)


{'query': 'What are travel allowance for E3 grade employees?', 'result': 'The travel allowance for E3 grade employees is lower of actual cost or Air (Economy Class)/2nd AC Train fare/Bus fare.'}
